In [1]:
import cantools
import csv
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import math

In [2]:
jarsPackages = "com.datastax.spark:spark-cassandra-connector_2.12:3.2.0"

In [3]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('Batch Bus Location from Sensor EV-BUS') \
                    .config("spark.jars.packages", jarsPackages) \
                    .config("spark.cassandra.connection.host", "10.184.0.2") \
                    .config('spark.cassandra.auth.username','cassandra') \
                    .config('spark.cassandra.auth.password','cassandra') \
                    .config('spark.cassandra.connection.keepAliveMS','3600000') \
                    .getOrCreate()

:: loading settings :: url = jar:file:/home/macbook/.local/share/virtualenvs/spark-processor-YU7I_jhp/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/macbook/.ivy2/cache
The jars for the packages stored in: /home/macbook/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0b144d4-3400-4676-a23b-0ffe324165cc;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central

22/11/28 00:03:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
csv = spark.read.option("header", "true").csv("DA1-12-13 (2).csv")
csv.printSchema()
csv.show(truncate=False)

root
 |-- Bus: string (nullable = true)
 |-- No: string (nullable = true)
 |-- Time (abs): string (nullable = true)
 |-- State: string (nullable = true)
 |-- ID (hex): string (nullable = true)
 |-- DLC: string (nullable = true)
 |-- Data (hex): string (nullable = true)
 |-- ASCII: string (nullable = true)

+----------------------------+------+------------+------+--------+---+-----------------------+--------+
|Bus                         |No    |Time (abs)  |State |ID (hex)|DLC|Data (hex)             |ASCII   |
+----------------------------+------+------------+------+--------+---+-----------------------+--------+
|USB-to-CAN V2 compact  CAN-1|18,878|00:00:49.554|  E   | CF00203|8  |CD FF FE FF FF FF FF FF|........|
|USB-to-CAN V2 compact  CAN-1|18,879|00:00:49.555|  E   |18FEBF0B|8  |4F 12 7D 7D 7D 7D FF FF|O.}}}}..|
|USB-to-CAN V2 compact  CAN-1|18,880|00:00:49.565|  E   | 8FE6E0B|8  |56 12 54 12 2C 12 38 12|V.T.,.8.|
|USB-to-CAN V2 compact  CAN-1|18,881|00:00:49.565|  E   | CF00203|8 

In [5]:
def convertToFloat(dictData):
   return dict([key, float(value)]
               for key, value in dictData.items())

In [6]:
dbc = cantools.database.load_file('./j1939.dbc')
dbc.add_dbc_file('./iso.dbc')
# msg=dbc.decode_message(0xCF002FE,bytes.fromhex('CD FF FE FF FF FF FF FF'),decode_choices=False)
# query = dbc.decode_message(419351294,bytes.fromhex('C0 FF 00 00 00 00 FF FF'))
# print(query)
# query = query.get_message_by_frame_id(DBC_ID)
# print(query)
# query = parseMessageNameByID('CF00203')
# print(msg)

Overwriting message 'LC' with 'LCMD' in the frame id to message dictionary because they have identical masked frame ids 0xcfe41fe.
Overwriting message 'GPV4' with 'GPV4' in the name to message dictionary.
Overwriting message 'GPV4' with 'GPV4' in the frame id to message dictionary because they have identical masked frame ids 0x1807fefe.
Overwriting message 'ECUID' with 'ECUID' in the name to message dictionary.
Overwriting message 'ECUID' with 'ECUID' in the frame id to message dictionary because they have identical masked frame ids 0x18fdc5fe.
Overwriting message 'LD' with 'LD' in the name to message dictionary.
Overwriting message 'LD' with 'LD' in the frame id to message dictionary because they have identical masked frame ids 0x18fe40fe.
Overwriting message 'SOFT' with 'SOFT' in the name to message dictionary.
Overwriting message 'SOFT' with 'SOFT' in the frame id to message dictionary because they have identical masked frame ids 0x18fedafe.
Overwriting message 'TD' with 'TD' in the

In [7]:
# DECODED_MESSAGE = []
LONG_MESSAGE = {}

def parser(ID_HEX,DLC,DATA_HEX_STR):
        ID_HEX = int(ID_HEX, 16)
        DATA_LEN = int(DLC)

        PRIORITY = ID_HEX & (0b00011100 << 24)
        RESERVED = ID_HEX & (0b00000010 << 24)
        DATA_PAGE = ID_HEX & (0b00000001 << 24)
        PDU_FORMAT = ID_HEX & (0b11111111 << 16)
        PDU_SPECIFIC = ID_HEX & (0b11111111 << 8)
        SOURCE_ADDRESS = ID_HEX & (0b11111111 << 0)

        PGN = RESERVED | DATA_PAGE | PDU_FORMAT | PDU_SPECIFIC
        DBC_ID = PRIORITY | PGN | 0xFE
        try:
            currMsg = dbc.get_message_by_frame_id(DBC_ID)
            try:
    #             DECODED_MESSAGE.append(currMsg.decode(bytes.fromhex(DATA_HEX_STR)))
                outdata = dbc.decode_message(DBC_ID,bytes.fromhex(DATA_HEX_STR),decode_choices=False)
                return convertToFloat(outdata)
            except:
                if currMsg.frame_id in LONG_MESSAGE.keys():
                    LONG_MESSAGE[currMsg.frame_id] = f"{LONG_MESSAGE[currMsg.frame_id]}{DATA_HEX_STR}"
                    try:
                        newOutdata = dbc.decode_message(DBC_ID,bytes.fromhex(LONG_MESSAGE[currMsg.frame_id]),decode_choices=False)
                        del LONG_MESSAGE[currMsg.frame_id]
                        return convertToFloat(newOutdata)
                    except:
                        pass
                else:
                    LONG_MESSAGE[currMsg.frame_id] = DATA_HEX_STR
        except:
            pass

In [8]:
# tempdict = {}

# result = []

In [9]:
# def parsing_asd(idhex, datahex):
#     idhex = idhex[0:len(idhex)-2]
#     idhex = idhex + "FE"
#     idhex = idhex.strip()
#     outdata = None
#     try:
#         outdata = decodeee(idhex, datahex)
#     except:
#         try:
#             newhex = idhex[0:len(idhex) - 4] + "FE" + idhex[len(idhex) - 2:len(idhex)]
#             outdata = decodeee(newhex, datahex)
#         except:
#             pass
#     if(outdata):
#         for key,value in outdata.items():
#             if not isinstance(outdata[key], int) and not isinstance(outdata[key], float) and not isinstance(outdata[key], str):
#                 outdata[key] = str(outdata[key])      
#         return outdata

In [10]:
# def decodeee(idhex, datahex):
#     ih = int("0x" + idhex, 16)
#     curmsg = dbc.get_message_by_frame_id(ih)
#     if curmsg.length > 8:
#         if idhex in tempdict:
#             tempdict[idhex] += datahex
#         else:
#             tempdict[idhex] = datahex
            
#         if len(tempdict[idhex]) / 2.875 >= curmsg.length:
#             outdata = dbc.decode_message(ih, bytes.fromhex(tempdict[idhex]),decode_choices=False)
#             outdata = convertToFloat(outdata)
#             tempdict.pop(idhex)
#             return outdata
#     else:
#         outdata = dbc.decode_message(ih, bytes.fromhex(datahex),decode_choices=False)
#         outdata = convertToFloat(outdata)
#         return outdata

In [11]:
# def parseMessageNameByID(idhex):
#     idhex = idhex[0:len(idhex)-2]
#     idhex = idhex + "FE"
#     idhex = idhex.strip()
#     try:
#         ih = int("0x" + idhex, 16)
#         curmsg = dbc.get_message_by_frame_id(ih)
#         return curmsg.name
#     except:
#         try:
#             newhex = idhex[0:len(idhex) - 4] + "FE" + idhex[len(idhex) - 2:len(idhex)]
#             ih = int("0x" + newhex, 16)
#             curmsg = dbc.get_message_by_frame_id(ih)
#             return curmsg.name
#         except:
#             pass

In [12]:
def parseMessageName(ID_HEX):
    ID_HEX = int(ID_HEX, 16)
    PRIORITY = ID_HEX & (0b00011100 << 24)
    RESERVED = ID_HEX & (0b00000010 << 24)
    DATA_PAGE = ID_HEX & (0b00000001 << 24)
    PDU_FORMAT = ID_HEX & (0b11111111 << 16)
    PDU_SPECIFIC = ID_HEX & (0b11111111 << 8)
    
    PGN = RESERVED | DATA_PAGE | PDU_FORMAT | PDU_SPECIFIC
    DBC_ID = PRIORITY | PGN | 0xFE
    
    try:
        currMsg = dbc.get_message_by_frame_id(DBC_ID)
        try: 
            return currMsg.name
        except:
               pass
    except:
        pass

In [13]:
parserCan = udf(lambda m,n,o: parser(m,n,o),MapType(StringType(),DoubleType()))

In [14]:
parserMessageName = udf(lambda m: parseMessageName(m))

In [15]:
parser = csv.withColumn("value", parserCan('ID (hex)','DLC','Data (hex)')) \
        .withColumn('message_name',when(parserMessageName('ID (hex)').isNull(),'null')
                    .otherwise(parserMessageName('ID (hex)'))) \
        .withColumn('timestamp',to_timestamp(col('Time (abs)'))) \
        .withColumn('bus_id', lit('bus_34')) \
        .withColumnRenamed('ID (hex)','can_id') \
        .withColumnRenamed('Data (hex)','data') \
        .withColumnRenamed('DLC','dlc') \
        .drop('Bus','No','State','ASCII','Time (abs)')

# keysDF = parser.select(explode(map_keys(parser.value))).distinct()
# # keysDF.show()
# keysList = keysDF.rdd.map(lambda x:x[0]).collect()
# keyCols = list(map(lambda x: col("value").getItem(x).alias(str(x)), keysList))
# query = parser.select('bus_id','MessageName','timestamp', *keyCols).show()

In [16]:
parser.show(vertical=True,truncate=False)
parser.printSchema()

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 can_id       |  CF00203                                                                                                                                                                                                                                                                                                                               

In [17]:
cassandra = parser.selectExpr('bus_id','message_name','can_id','dlc','data','value','timestamp')  \
            .write \
            .format("org.apache.spark.sql.cassandra") \
            .options(table='sensor_data_bus', keyspace="busev") \
            .mode('append')\
            .save()